In [1]:
import pathpy as pp

This notebook shows how to use `pathpy 2.0` to estimate a multi-order model based on origin/destination passenger data, using the example of the London Tube. The data needed to run this notebook can be obtained from the [Rolling Origin Destination Survey (RODS)](https://data.london.gov.uk/dataset/tfl-rolling-origin-and-destination-survey) available from the "Transport for London".

We first read the topology of the tube network provided in the file `tube.edges`. In this file, every node is a Tube station given in terms of a unique ID. In pathpy, the key abstraction is a path and every edge is simply a path of length one. Thus, we can simply read the edge file as a (trivial) example of a path data set. 

In [2]:
p = pp.Paths.readEdges('data/tube.edges', separator=' ', undirected=True)

2018-02-09 15:30:05 [Severity.INFO]	Reading edge data ... 
2018-02-09 15:30:05 [Severity.INFO]	Calculating sub path statistics ... 
2018-02-09 15:30:05 [Severity.INFO]	finished.


Since the origin/destination contain the *names* of Tube stations rather than node IDs, we first need to map the node IDs in the network above to station names. This mapping is provided in the file `tube.nodes`. We read it and generate a mapping dictionary.

In [3]:
mapping = {}
with open('data/tube.nodes', 'r') as f:
    line = f.readline()
    while line:
        fields = line.rstrip().split(';')
        mapping[fields[0]] = fields[1]
        line = f.readline()

We can now use pathpy's projectPaths method to apply the bijective mapping from node IDs to station names: 

In [4]:
p = p.projectPaths(mapping)

To calculate shortest paths, we need a first-order graph representation of the London Tube. We can do this by generating a `HigherOrderNetwork` from the paths (which in our case contains edges). If we don't explicitly specify a higher 
order k, pathpy generates a first-order graph representation

In [5]:
network = pp.HigherOrderNetwork(p)

We next read the origin/destination statistics from the file `tube_od.csv`. We can do this using the respective `readFile` function in `pathpy`'s `OriginDestionationPath` class.

In [6]:
origin_destination = pp.PathExtraction.OriginDestinationPaths.readFile('data/tube_od.csv', separator=';')

2018-02-09 15:30:05 [Severity.INFO]	Reading origin/destination statistics from file ...
2018-02-09 15:30:05 [Severity.INFO]	Finished.


Now we have everything needed to extract path statistics. The key idea is that for each origin-destination pair that was observed w times, we assume that passengers traveled along the shortest paths in the network. Note that there can be multiple different shortest paths between the same pair of nodes that have identical length. In this case, we equally distribute integer numbers of observations among all possible shortest paths, i.e. if there are three shortest path `O-A-D`, `O-B-D`, `O-C-D` between origin O and destination D, and we observe the origin-destination pair `O -> D` 7 times, we assume that O-A-D was taken 3 times, while `O-B-D` and `O-B-C` were taken 2 times each. If needed, we can set the parameter `distribute_weight=False` to switch off this behavior, instead randomly picking a single shortest path and assigning all weight to this single path.

In [7]:
paths = pp.PathExtraction.OriginDestinationPaths.extract(origin_destination, network)
print(paths)

2018-02-09 15:30:05 [Severity.INFO]	Calculating shortest paths in higher-order network (k = 1) ...
2018-02-09 15:30:21 [Severity.INFO]	finished.
2018-02-09 15:30:21 [Severity.INFO]	Starting origin destination path calculation ...
2018-02-09 15:37:12 [Severity.INFO]	finished.
Total path count: 		4295731.0 
[Unique / Sub paths / Total]: 	[67015.0 / 177514561.0 / 181810292.0]
Nodes:				268 
Edges:				646
Max. path length:		35
Avg path length:		6.749005931702893 
Paths of length k = 0		0.0 [ 0.0 / 33287645.0 / 33287645.0 ]
Paths of length k = 1		200051.0 [ 598.0 / 28791863.0 / 28991914.0 ]
Paths of length k = 2		347599.0 [ 976.0 / 24348584.0 / 24696183.0 ]
Paths of length k = 3		461483.0 [ 1526.0 / 20139020.0 / 20600503.0 ]
Paths of length k = 4		474083.0 [ 2161.0 / 16378339.0 / 16852422.0 ]
Paths of length k = 5		486664.0 [ 2854.0 / 13079160.0 / 13565824.0 ]
Paths of length k = 6		420703.0 [ 3583.0 / 10332606.0 / 10753309.0 ]
Paths of length k = 7		353147.0 [ 4115.0 / 8074311.0 / 8427458.

We can now create a multi-order model from the generated path statistics 

In [8]:
m = pp.MultiOrderModel(paths, maxOrder=7)

2018-02-09 15:37:13 [Severity.INFO]	Generating 0-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 1-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 2-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 3-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 4-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 5-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 6-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	Generating 7-th order network layer ...
2018-02-09 15:37:13 [Severity.INFO]	finished.


We finally use a representation learning algorithm (presented in the [KDD'17 paper](http://dl.acm.org/citation.cfm?id=3098145) below) to estimate the optimal number of layers of higher-order graphical models needed to model this data set.

In [9]:
kopt = m.estimateOrder(paths, maxOrder=6)
print('Optimal order of multi-order graph model of London Tube network = ' + str(kopt))

2018-02-09 15:38:14 [Severity.INFO]	Likelihood ratio test for K_opt = 2, x = 31852224.98864612
2018-02-09 15:38:14 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 1622
2018-02-09 15:38:14 [Severity.INFO]	Likelihood ratio test, p = 0.0
2018-02-09 15:39:24 [Severity.INFO]	Likelihood ratio test for K_opt = 3, x = 1002708.4119753093
2018-02-09 15:39:24 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 5503
2018-02-09 15:39:24 [Severity.INFO]	Likelihood ratio test, p = 0.0
2018-02-09 15:40:43 [Severity.INFO]	Likelihood ratio test for K_opt = 4, x = 404338.1268621832
2018-02-09 15:40:43 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 19157
2018-02-09 15:40:43 [Severity.INFO]	Likelihood ratio test, p = 0.0
2018-02-09 15:42:16 [Severity.INFO]	Likelihood ratio test for K_opt = 5, x = 369247.02671541274
2018-02-09 15:42:16 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 66615
2018-02-09 15:42:16 [Severity.INFO]	Likelihood ratio test, p = 0.0
2018-02-09 15:44:07 [Severity.INFO]	Likelihood ratio

The result shows that **we need six layers of higher-order graphical models to understand passenger paths in the London Tube**. This highlights that a simple **network model of the London Tube yields misleading results**.

For more information on the implications for dynamical processes or node centrality measures please refer to the following recent papers:

- I Scholtes: [When is a network a network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

- I Scholtes, N Wider, A Garas: [Higher-Order Aggregate Networks in the Analysis of Temporal Networks: Path structures and centralities](http://link.springer.com/article/10.1140%2Fepjb%2Fe2016-60663-0), The European Physical Journal B, 89:61, March 2016 

- I Scholtes, N Wider, R Pfitzner, A Garas, CJ Tessone, F Schweitzer: [Causality-driven slow-down and speed-up of diffusion in non-Markovian temporal networks](http://www.nature.com/ncomms/2014/140924/ncomms6024/full/ncomms6024.html), Nature Communications, 5, September 2014 